In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.signal as signal
#from calipertron import *

In [ ]:
# ADC sampling rate in Hz
sampling_rate = 251.748 * 1e3

pdm_drive_rate = 100 * 1e3

filename = "../frontend/100kHz_drive.txt"
#filename = "../frontend/100kHz_drive_one_signal.txt"

raw = pd.read_csv(filename, header = None, names = ["adc"])
raw['t'] = range(0, len(raw))
raw

In [ ]:
offset = 10000
n = 100000
d = raw.to_numpy()[offset:,][:n, 0]
sns.lineplot(x = range(n), y = d)
plt.show()

In [ ]:
## Examine the PDM signal we're generating
bsrr = [
    0b00000000010101010000000010101010,
    0b00000000010101010000000010101010,
    0b00000000011010100000000010010101,
    0b00000000011010100000000010010101,
    0b00000000010101010000000010101010,
    0b00000000100101010000000001101010,
    0b00000000011010100000000010010101,
    0b00000000011010100000000010010101,
    0b00000000010101010000000010101010,
    0b00000000100101010000000001101010,
    0b00000000011010100000000010010101,
    0b00000000011010100000000010010101,
    0b00000000100101010000000001101010,
    0b00000000100101010000000001101010,
    0b00000000010110100000000010100101,
    0b00000000011010100000000010010101,
    0b00000000100101010000000001101010,
    0b00000000100101010000000001101010,
    0b00000000010110100000000010100101,
    0b00000000010110100000000010100101,
    0b00000000100101010000000001101010,
    0b00000000101001010000000001011010,
    0b00000000010110100000000010100101,
    0b00000000010110100000000010100101,
    0b00000000100101010000000001101010,
    0b00000000101001010000000001011010,
    0b00000000010110100000000010100101,
    0b00000000010110100000000010100101,
    0b00000000101001010000000001011010,
    0b00000000101001010000000001011010,
    0b00000000010101100000000010101001,
    0b00000000010110100000000010100101,
    0b00000000101001010000000001011010,
    0b00000000101001010000000001011010,
    0b00000000010101100000000010101001,
    0b00000000010101100000000010101001,
    0b00000000101001010000000001011010,
    0b00000000101010010000000001010110,
    0b00000000010101100000000010101001,
    0b00000000010101100000000010101001,
    0b00000000101001010000000001011010,
    0b00000000101010010000000001010110,
    0b00000000010101100000000010101001,
    0b00000000010101100000000010101001,
    0b00000000101010010000000001010110,
    0b00000000101010010000000001010110,
    0b00000000010101010000000010101010,
    0b00000000010101100000000010101001,
    0b00000000101010010000000001010110,
    0b00000000101010010000000001010110,
    0b00000000010101010000000010101010,
    0b00000000010101010000000010101010,
    0b00000000101010010000000001010110,
    0b00000000101010100000000001010101,
    0b00000000010101010000000010101010,
    0b00000000010101010000000010101010,
    0b00000000101010010000000001010110,
    0b00000000101010100000000001010101,
    0b00000000010101010000000010101010,
    0b00000000010101010000000010101010,
    0b00000000101010100000000001010101,
    0b00000000101010100000000001010101,
    0b00000000010101010000000010101010,
    0b00000000010101010000000010101010,
    0b00000000101010100000000001010101,
    0b00000000101010100000000001010101,
    0b00000000100101010000000001101010,
    0b00000000010101010000000010101010,
    0b00000000101010100000000001010101,
    0b00000000101010100000000001010101,
    0b00000000100101010000000001101010,
    0b00000000100101010000000001101010,
    0b00000000101010100000000001010101,
    0b00000000011010100000000010010101,
    0b00000000100101010000000001101010,
    0b00000000100101010000000001101010,
    0b00000000101010100000000001010101,
    0b00000000011010100000000010010101,
    0b00000000100101010000000001101010,
    0b00000000100101010000000001101010,
    0b00000000011010100000000010010101,
    0b00000000011010100000000010010101,
    0b00000000101001010000000001011010,
    0b00000000100101010000000001101010,
    0b00000000011010100000000010010101,
    0b00000000011010100000000010010101,
    0b00000000101001010000000001011010,
    0b00000000101001010000000001011010,
    0b00000000011010100000000010010101,
    0b00000000010110100000000010100101,
    0b00000000101001010000000001011010,
    0b00000000101001010000000001011010,
    0b00000000011010100000000010010101,
    0b00000000010110100000000010100101,
    0b00000000101001010000000001011010,
    0b00000000101001010000000001011010,
    0b00000000010110100000000010100101,
    0b00000000010110100000000010100101,
    0b00000000101010010000000001010110,
    0b00000000101001010000000001011010,
    0b00000000010110100000000010100101,
    0b00000000010110100000000010100101,
    0b00000000101010010000000001010110,
    0b00000000101010010000000001010110,
    0b00000000010110100000000010100101,
    0b00000000010101100000000010101001,
    0b00000000101010010000000001010110,
    0b00000000101010010000000001010110,
    0b00000000010110100000000010100101,
    0b00000000010101100000000010101001,
    0b00000000101010010000000001010110,
    0b00000000101010010000000001010110,
    0b00000000010101100000000010101001,
    0b00000000010101100000000010101001,
    0b00000000101010100000000001010101,
    0b00000000101010010000000001010110,
    0b00000000010101100000000010101001,
    0b00000000010101100000000010101001,
    0b00000000101010100000000001010101,
    0b00000000101010100000000001010101,
    0b00000000010101100000000010101001,
    0b00000000010101010000000010101010,
    0b00000000101010100000000001010101,
    0b00000000101010100000000001010101,
    0b00000000010101100000000010101001,
    0b00000000010101010000000010101010,
    0b00000000101010100000000001010101,
    0b00000000101010100000000001010101,
    0b00000000010101010000000010101010,
    0b00000000010101010000000010101010,
    0b00000000011010100000000010010101,
    0b00000000101010100000000001010101,   
]

import numpy as np

# Convert BSRR values to GPIO states
def bsrr_to_gpio_states(bsrr_values):
    num_samples = len(bsrr_values)
    gpio_states = np.zeros((num_samples, 16), dtype=int)
    
    for i, bsrr in enumerate(bsrr_values):
        set_bits = bsrr & 0xFFFF
        reset_bits = (bsrr >> 16) & 0xFFFF
        
        if i > 0:
            gpio_states[i] = gpio_states[i-1]
        
        for j in range(16):
            if set_bits & (1 << j):
                gpio_states[i, j] = 1
            elif reset_bits & (1 << j):
                gpio_states[i, j] = 0
    
    return gpio_states

# Generate GPIO state matrix
gpio_matrix = bsrr_to_gpio_states(bsrr)

# Print shape of the resulting matrix
print(f"GPIO matrix shape: {gpio_matrix.shape}")

# Optionally, visualize the first few rows of the matrix
print("\nFirst 5 rows of the GPIO matrix:")
print(gpio_matrix[:5])


In [ ]:
def pdm_to_analog(pdm_signal, sampling_rate, cutoff_freq):
    # Design the low-pass filter
    nyquist_rate = 0.5 * sampling_rate
    normal_cutoff = cutoff_freq / nyquist_rate
    b, a = signal.butter(4, normal_cutoff, btype='low', analog=False)
    
    # Apply the filter
    analog_signal = signal.filtfilt(b, a, pdm_signal)
    
    return analog_signal

pdm_single = np.tile(gpio_matrix[:,2], 10)

# Example usage:
cutoff_freq = 1000  # Adjust based on your signal characteristics
analog_signal = pdm_to_analog(pdm_single, pdm_drive_rate, cutoff_freq)

# Plotting
t = np.arange(len(pdm_single)) / sampling_rate
plt.figure(figsize=(12, 6))
plt.step(t, pdm_single, 'b-', where='post', label='PDM Signal', alpha=0.5)
plt.plot(t, analog_signal, 'r-', label='Reconstructed Analog Signal')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('PDM to Analog Conversion')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:

# Find the dominant frequency of analog_signal
from scipy.fft import fft, fftfreq

# Perform FFT
n = len(analog_signal)
fft_result = fft(analog_signal)
frequencies = fftfreq(n, 1/pdm_drive_rate)

# Find the index of the maximum amplitude in the positive frequency range
positive_freq_range = frequencies > 0
max_amplitude_index = np.argmax(np.abs(fft_result[positive_freq_range]))

# Get the dominant frequency
dominant_frequency = frequencies[positive_freq_range][max_amplitude_index]

print(f"The dominant frequency is approximately {dominant_frequency:.2f} Hz")

# Optional: Plot the frequency spectrum
plt.figure(figsize=(12, 6))
# Limit the frequency range to 10 kHz
freq_limit = 10000
freq_mask = (frequencies[positive_freq_range] <= freq_limit)
plt.plot(frequencies[positive_freq_range][freq_mask], np.abs(fft_result[positive_freq_range][freq_mask]))
plt.xlim(0, freq_limit)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.title('Frequency Spectrum of the Analog Signal')
plt.axvline(x=dominant_frequency, color='r', linestyle='--', label=f'Dominant Frequency: {dominant_frequency:.2f} Hz')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Apply a high-pass filter to remove low frequency noise
from scipy import signal

def apply_high_pass_filter(data, sampling_rate=sampling_rate, cutoff=1000, order=6):
    """
    Apply a high-pass filter to remove low frequency noise.
    
    Parameters:
    data (array-like): Input data to be filtered
    fs (int): Sampling frequency in Hz (default: 100000)
    cutoff (int): Cutoff frequency for high-pass filter in Hz (default: 1000)
    order (int): Filter order (default: 6)
    
    Returns:
    array-like: Filtered data
    """
    # Compute the Nyquist frequency
    nyquist = 0.5 * sampling_rate

    # Compute the normalized cutoff frequency
    normalized_cutoff = cutoff / nyquist

    # Design the high-pass filter
    b, a = signal.butter(order, normalized_cutoff, btype='high', analog=False)

    # Apply the filter to the data
    filtered_data = signal.filtfilt(b, a, data)
    
    return filtered_data

# Apply the high-pass filter to the data
d_filtered = apply_high_pass_filter(d)

# Define the number of points to plot
num_points = 5000

# Plot the original and filtered data for comparison using seaborn
plt.figure(figsize=(12, 6))
sns.lineplot(x=range(num_points), y=d[:num_points], label='Original')
sns.lineplot(x=range(num_points), y=d_filtered[:num_points], label='Filtered')
plt.xlabel('Sample')
plt.ylabel('ADC Value')
plt.title(f'Original vs High-Pass Filtered Data (First {num_points} Points)')
plt.legend()
plt.show()

# Create a zoomed-in plot of d_filtered
offset = 1000
length = 1500
plt.figure(figsize=(12, 6))
sns.lineplot(x=range(offset, offset + length), y=d_filtered[offset:(offset + length)])
plt.xlabel('Sample')
plt.ylabel('ADC Value')
plt.title('Zoomed-in View of Filtered Data')
plt.show()



In [ ]:
# Calculate rolling window of absolute amplitude
window_size = 20  # Adjust this value as needed
rolling_amplitude = pd.Series(np.abs(d_filtered)).rolling(window=window_size, center=True).mean()

# Plot the rolling window of absolute amplitude
plt.figure(figsize=(12, 6))
plt.plot(rolling_amplitude[offset:(offset + length)])
plt.xlabel('Sample')
plt.ylabel('Absolute Amplitude')
plt.title(f'Rolling Window of Absolute Amplitude (Window Size: {window_size})')
plt.show()


In [ ]:
# Fit a single sinusoidal signal to rolling_amplitude
from scipy.optimize import curve_fit

def sinusoidal(x, A, f, phi, C):
    return A * np.sin(2 * np.pi * f * x + phi) + C

# Prepare data for fitting
y = rolling_amplitude[offset:(offset + length)]
x = np.arange(len(y))


# Initial guess for parameters (A, f, phi, C)
initial_guess = [np.std(y), 1 / (len(x) / 10), 0, np.mean(y)]

# Perform the curve fitting
params, _ = curve_fit(sinusoidal, x, y, p0=initial_guess)

params = [np.std(y), 1 / (len(x) / 14.5), 0, np.mean(y)]
# Extract the fitted parameters
A, f, phi, C = params


# Calculate the frequency in Hz
freq_hz = f * sampling_rate

print(f"Fitted frequency: {freq_hz:.2f} Hz")
print(f"Phase offset: {phi:.2f} radians")

# Plot the original data and the fitted curve
plt.figure(figsize=(12, 6))
plt.plot(x, y, label='Original data')
plt.plot(x, sinusoidal(x, *params), label='Fitted curve', color='r')
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.title('Sinusoidal Fit to Rolling Amplitude')
plt.legend()
plt.show()


In [ ]:
# Perform FFT on the filtered data
rolling = rolling_amplitude[offset:(offset + 50000)]
n = len(rolling)
fft_result = np.fft.fft(rolling)
frequencies = np.fft.fftfreq(n, 1/sampling_rate)

# Calculate the power spectrum
power_spectrum = np.abs(fft_result)**2

# Plot the power spectrum
import plotly.graph_objects as go

# Find the index corresponding to 10 kHz
max_freq_index = np.argmax(frequencies >= 10000)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=frequencies[:max_freq_index],
    y=power_spectrum[:max_freq_index],
    mode='lines',
    name='Power Spectrum'
))

fig.update_layout(
    title='Power Spectrum of Filtered Data (0-10 kHz)',
    xaxis_title='Frequency (Hz)',
    yaxis_title='Power',
    yaxis_type='log',
    xaxis_range=[0, 10000],
    width=800,
    height=500,
)

fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=True)

fig.show()